In [ ]:
import re
import pandas as pd
import csv
import numpy as np
from collections import Counter
import nltk
import random
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')
import gensim
from gensim.models import Word2Vec

In [ ]:
import tensorflow as tf
import math
import os
import time
import numpy as np
import pandas as pd
import random
import statistics
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential, regularizers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Input, LSTM, Dropout, TimeDistributed, RepeatVector, Bidirectional
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredLogarithmicError
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy, Precision, AUC, Recall
from sklearn.metrics import silhouette_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_outlier = pd.read_csv('/content/drive/MyDrive/thesis/val/raw_val_all.csv', sep=',')

In [ ]:
true_labels = df_outlier.label

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/thesis/train/reduce_pca_cv.csv', sep = ',')
x_out = pd.read_csv('/content/drive/MyDrive/thesis/val/val-reduce_cv_pca.csv', sep = ',')

In [ ]:
test = df[:int(split_len)]
train = df[int(split_len):]

In [ ]:
pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184097 sha256=12b61153fd546778542bfd7558a87827e8a77a1afcb0404f3ccd6bba475d734c
  Stored in directory: /root/.cache/pip/wheels/83/55/6b/552e083cf5509c0afe808b76cf434f1be284d01a112623bd37
Successfully built pyod


In [ ]:
x_train = train.values.reshape(train.shape[0], 1, train.shape[1])
x_test = test.values.reshape(test.shape[0], 1,train.shape[1])

x_out_val = x_out.values.reshape(x_out.shape[0], 1,x_out.shape[1])

#define the model

def autoencoder_model(X):

  inputs = Input(shape=(X.shape[1], X.shape[2]))
  L1 = Bidirectional(LSTM(128, activation='relu', return_sequences=True,
              kernel_regularizer=regularizers.l2(0.00)))(inputs)
  L2 = Bidirectional(LSTM(64, activation='relu', return_sequences=False))(L1)
  L3 = RepeatVector(X.shape[1])(L2)
  L4 = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(L3)
  L5 = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(L4)
  output = TimeDistributed(Dense(X.shape[2]))(L5)
  model = Model(inputs=inputs, outputs=output)
  return model


# create the autoencoder model
model = autoencoder_model(x_train)
model.compile(optimizer='adam', loss=['mse'],
              metrics=['mae', 'acc', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model.summary()

nb_epochs = 10
batch_size = 1000
history = model.fit(x_train, x_train, epochs=nb_epochs, batch_size=batch_size,
                    validation_split=0.15).history

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 25)]           0         
                                                                 
 bidirectional (Bidirectiona  (None, 1, 256)           157696    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 repeat_vector (RepeatVector  (None, 1, 128)           0         
 )                                                               
                                                                 
 bidirectional_2 (Bidirectio  (None, 1, 128)           98816     
 nal)                                                        

In [ ]:
# calculate the loss on the test set
X_pred = model.predict(x_out_val)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=x_out.columns)
X_pred.index = x_out.index

scored = pd.DataFrame(index=x_out.index)
Xtest = x_out.values.reshape(x_out.shape[0], x_out.shape[1])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtest), axis = 1)
scored['Threshold'] = 0.1
scored['Anomaly'] = np.where(scored['Loss_mae'] > scored['Threshold'], -1, 1)
scored.head()
test_pred = scored.Anomaly
from sklearn.metrics import classification_report
print(classification_report(true_labels, test_pred))

3188/3188 [==============================] - 10s 3ms/step
              precision    recall  f1-score   support

          -1       0.80      0.93      0.86      3737
           1       1.00      0.99      0.99     98263

    accuracy                           0.99    102000
   macro avg       0.90      0.96      0.93    102000
weighted avg       0.99      0.99      0.99    102000



In [ ]:
from pyod.models.copod import COPOD
from pyod.models.ecod import ECOD
models = [COPOD(contamination=0.01), ECOD(contamination=0.005), IsolationForest(n_estimators=200, contamination = 0.005)]

In [ ]:
# Train the individual models
for model in models:
    model.fit(train)

# Make predictions using each individual model
predictions = []
for model in models:
    model_predictions = model.predict(x_out)
    if (model_predictions == 1).any():

      model_predictions[model_predictions == 1] = -1

    if (model_predictions == 0).any():
      model_predictions[model_predictions == 0] = 1
    predictions.append(model_predictions)

predictions.append(test_pred)

# Perform majority voting to obtain the final ensemble prediction
final_predictions = []
for i in range(len(x_out)):
    votes = [predictions[j][i] for j in range(4)]
    majority_vote = max(set(votes), key=votes.count)
    final_predictions.append(majority_vote)

In [ ]:
Counter(final_predictions)

Counter({1: 97539, -1: 4461})

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, final_predictions))

              precision    recall  f1-score   support

          -1       0.82      0.98      0.89      3737
           1       1.00      0.99      1.00     98263

    accuracy                           0.99    102000
   macro avg       0.91      0.99      0.94    102000
weighted avg       0.99      0.99      0.99    102000



TF+`pca

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/thesis/train/reduce_pca_tf.csv', sep=',')
x_out = pd.read_csv('/content/drive/MyDrive/thesis/val/val-reduce_pca_tf.csv', sep=',')

In [ ]:
test = df[:int(split_len)]
train = df[int(split_len):]

In [ ]:
from pyod.models.copod import COPOD
from pyod.models.ecod import ECOD
models = [COPOD(contamination=0.02), ECOD(contamination=0.009), IsolationForest(n_estimators=200, contamination=0.015)]
#COPOD(contamination=0.01)

In [ ]:
pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184097 sha256=64bd9ba488446db4ef8ea33d183d2b8b3b971b208ecdc1fca3a3de043e84892a
  Stored in directory: /root/.cache/pip/wheels/83/55/6b/552e083cf5509c0afe808b76cf434f1be284d01a112623bd37
Successfully built pyod


In [ ]:
# Train the individual models
for model in models:
    model.fit(train)

# Make predictions using each individual model
predictions = []
for model in models:
    model_predictions = model.predict(x_out)
    if (model_predictions == 1).any():

      model_predictions[model_predictions == 1] = -1

    if (model_predictions == 0).any():
      model_predictions[model_predictions == 0] = 1
    predictions.append(model_predictions)



In [ ]:
# Perform majority voting to obtain the final ensemble prediction
final_predictions = []
for i in range(len(x_out)):
    votes = [predictions[j][i] for j in range(3)]
    majority_vote = max(set(votes), key=votes.count)
    final_predictions.append(majority_vote)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, final_predictions))

              precision    recall  f1-score   support

          -1       0.82      1.00      0.90      3737
           1       1.00      0.99      1.00     98263

    accuracy                           0.99    102000
   macro avg       0.91      1.00      0.95    102000
weighted avg       0.99      0.99      0.99    102000



In [ ]:

# Perform model averaging to obtain the final prediction
final_prediction = sum(predictions) / len(predictions)  # Average the predictions

# Convert probabilities to class labels if necessary
if isinstance(final_prediction[0], float):
    final_prediction = [1 if p > 0.276899 else -1 for p in final_prediction]

In [ ]:
threshold = 0.9999999 # Adjust the threshold as per your needs

# Convert probabilities to anomaly labels based on the adjusted threshold
final_prediction = [-1 if p >= threshold else 1 for p in final_prediction]